In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
sys.path.append('..')
import os
import ast
import pandas as pd

from tqdm.autonotebook import tqdm
tqdm.pandas()

import json

/var/folders/73/1_fd1jr14477j2fv2b70mpx00000gn/T/ipykernel_8324/440396935.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
from kst.data_sets.TextSequences import TextSequences
from kst.data_sets.TextSequencesCorpus import TextSequencesCorpus

In [4]:
from gensim.parsing.preprocessing import (
    strip_tags,
    strip_punctuation,
    strip_multiple_whitespaces,
    remove_stopwords,
    strip_short,
    strip_numeric,
    stem_text
)

In [5]:
GENSIM_CUSTOM_TEKENIZER = [
            strip_tags,
            strip_punctuation,
            strip_multiple_whitespaces,
            strip_numeric
        ]

# Load data

### Lines

In [6]:
lines_dict = {}
counter = 0
with open("/Users/lukaszmaczewski/Documents/Learning/DeepNeuralNetworks/keras/data/cornell_movie_dialog_corpus/movie_lines.txt", "r", encoding='iso-8859-1') as f:
    for line in tqdm(f.readlines()):
        line = line.split(" +++$+++ ")
        lines_dict[(line[2],line[0])] = line[4]
        counter += 1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 304713/304713 [00:00<00:00, 610546.86it/s]


### Create corpus of conversations 

In [7]:
tc = TextSequencesCorpus()
with open("/Users/lukaszmaczewski/Documents/Learning/DeepNeuralNetworks/keras/data/cornell_movie_dialog_corpus/movie_conversations.txt","r") as f:
    for line in tqdm(f.readlines()):
        line = line.split(" +++$+++ ")
        movie = line[2]
        conversation =ast.literal_eval(line[3])
        sequences = TextSequences()
        for line in conversation:
            sequences.add_sequence(lines_dict.get((movie, line)),GENSIM_CUSTOM_TEKENIZER)
            tc.add_sequences(sequences)
tc.prepare_token_to_index_mappings()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 83097/83097 [00:14<00:00, 5724.69it/s]


# Build data set of consecutive dialogue

In [8]:
input = []
output = []
for seq in tqdm(tc.collection_of_sequences):
    for seq_1, seq_2 in seq.get_two_consecutive_sequences():
        input += [[tc.get_index(i) for i in seq_1]]
        output += [[tc.get_index(i) for i in seq_2]]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 304713/304713 [00:13<00:00, 22571.62it/s]


In [9]:
corpus_df = pd.DataFrame.from_dict({'input_seq':input, 'output_seq':output})

# Store corpus

In [10]:
corpus_df.sample(frac = 1.0).to_pickle('/Users/lukaszmaczewski/Documents/Learning/DeepNeuralNetworks/keras/data/cornell_movie_dialog_corpus/corpus_df.pkl')

In [11]:
with open("/Users/lukaszmaczewski/Documents/Learning/DeepNeuralNetworks/keras/data/cornell_movie_dialog_corpus/token2index.json", "w") as outfile:
    outfile.write(json.dumps(tc._token2index))

In [12]:
with open("/Users/lukaszmaczewski/Documents/Learning/DeepNeuralNetworks/keras/data/cornell_movie_dialog_corpus/index2token.json", "w") as outfile:
    outfile.write(json.dumps(tc._index2token))